In [1]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher


In [2]:
def find_closest_name(name, other_names):
    checked = []
    for other_name in other_names:
        if other_name in checked:
            continue
        if(isinstance(other_name, float)):
            continue
        if name in other_name:
            return other_name

In [3]:
def find_most_similar_name(target_name, names_list, threshold=0.25):
    similarities = [(other_name, SequenceMatcher(None, target_name, other_name).ratio()) for other_name in names_list]
    most_similar_name, similarity_score = max(similarities, key=lambda x: x[1])
    
    if similarity_score >= threshold:
        print('found player '+most_similar_name+" with a score of "+str(similarity_score))
        return most_similar_name, similarity_score
    else:
        print("didn't find", target_name)
        return None, None

def find_player(player_name, club_name,  df, attributes):
    #first we filter by club name
    temp = df[df['Club Name'] == club_name]

    #Now we find the player
    sim_name, full_score = find_most_similar_name(player_name, temp['Full Name'])
    sim_nickname, short_score = find_most_similar_name(player_name, temp['Known As'])
    if sim_name: # there is a full name
        ## if there is a nickname we have to check
        if sim_nickname and full_score > short_score:
            return temp[temp['Full Name'] == sim_name].iloc[0][attributes]
        elif sim_nickname:
            return temp[temp['Known As'] == sim_nickname].iloc[0][attributes]
        return temp[temp['Full Name'] == sim_name].iloc[0][attributes]
    if sim_nickname: # if there is a nickname but no full name
        return temp[temp['Known As'] == sim_nickname].iloc[0][attributes]
    return None


def ratings_col(df, att_df):
    '''
    puts for every player their 'Overall', 'Age', 'Height(in cm)', 'Weight(in kg)'
    for each player i we will have the column
    HomePlayeri (if the player is on the home team)
    AwayPlayeri (if the player is on the away team)
    and we will use the home_team_name or away_team_name and the full name as the key (we will use find_player(player_name, club_name,  df, attributes))
    '''

    attributes = [ 'Overall']
    # Iterate through each player column
    for i in range(1, 12):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns or away_col in df.columns:
            # Add columns for home team players
            for att in attributes:
                df[home_col + "_" + att] = df.apply(
                    lambda row: 0 if pd.isna(row[home_col]) else
                    (find_player(row[home_col], row['home_team_name'], att_df, attributes)[att]
                    if find_player(row[home_col], row['home_team_name'], att_df, attributes) is not None else None),
                    axis=1
                )
            

            # Add columns for away team players
            for att in attributes:
                df[away_col + "_" + att] = df.apply(
                    lambda row: 0 if pd.isna(row[away_col]) else
                    (find_player(row[away_col], row['away_team_name'], att_df, attributes)[att]
                    if find_player(row[away_col], row['away_team_name'], att_df, attributes) is not None else None),
                    axis=1
                )
def replace_nas(df):
    '''
    gets df, goes to the Age column and replaces None with 18, and replaces Weight(in kg) and Height(in cm) with the mean, and overall with the min value
    '''

    # 'Dribbling Total', 'Pace Total', 'Defending Total', 'Shooting Total', 'Physicality Total', 'Passing Total' our new fatures
    attributes = ['Overall']
    # First, deal with the None values
    for i in range(1,12):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns:
            for att  in attributes: 
                non_zero_col = df[home_col+'_'+att].replace(0, None)
                df[home_col+'_'+att].fillna(non_zero_col.min())
        if away_col in df.columns:

            for att  in attributes: 
                non_zero_col = df[away_col+'_'+att].replace(0, None)
                df[away_col+'_'+att].fillna(non_zero_col.min())
           

    # Dealing with zero values
    for i in range(1, 12):
        home_col = f'HomePlayer{i}'
        away_col = f'AwayPlayer{i}'
        if home_col in df.columns:
            for att in attributes:
                df[home_col+"_"+att].replace(0, None, inplace=True)
        if away_col in df.columns:
            for att in attributes:
                df[away_col+"_"+att].replace(0, None, inplace=True)
          

def set_season_cols(columns, season):
    for c in season.columns:
        for att in ['Overall']:
            if att in c:
                print(c)
                columns.append(c)
def create_csv(df, columns, name):
    df[columns].to_csv(name)

def pipeline(matches_df, ratings_df):
    
    att_list = ['Overall']
    for att in att_list:
        if att not in ratings_df.columns:
            print('fix',att)
    

    # Sanity Check
    ratings_df = ratings_df[['Full Name', 'Club Name', 'Known As']+ att_list]
    ratings_col(matches_df, ratings_df)
    replace_nas(matches_df)
    final_cols = ['home_score', 'home_team_name', 'away_score', 'away_team_name','home_GD_prior', 'away_GD_prior', 'home_Points_prior', 'away_Points_prior',
                   'home_GD_form', 'home_Points_form', 'away_GD_form','away_Points_form', 'B365A', 'B365D', 'B365H',
                  'Matchweek']
    set_season_cols(final_cols, matches_df)
    return matches_df

In [4]:
leagues = ['laliga','seriea','bl']
years = ['1415','1516','1617', '1718','1819','1920','2021','2122','2223']

# #loading all the data to one data per 

for year in years:
    df = []
    for league in leagues:
        curr = pd.read_csv(league+"_data/"+league+year+"_form_bets.csv")
        if league == 'ucl':
            curr.rename(columns={'Matchweek' : "Stage"}, inplace=True)
        curr['league'] = league

        df.append(curr)
    curr_df = pd.concat(df,ignore_index=True)
    
    curr_df.drop(["Unnamed: 0.2","Unnamed: 0.1"], axis=1, inplace=True)
    curr_players = pd.read_csv('players_'+year[-2:]+'.csv')

    ### make sure we are only in eu


    ############ Now we will do the same thing we did in the players and matches pipeline but only take the fifa ratings
    curr_players.rename(columns={'long_name' : 'Full Name',
                          'short_name': "Known As", 'height_cm':'Height(in cm)',
                          'weight_kg' : 'Weight(in kg)',
                          'age' : 'Age','club_name' :"Club Name",
                          'overall' : 'Overall', 'dribbling' : 'Dribbling Total',
                          'pace' : 'Pace Total', 'defending' : "Defending Total" ,
                          "shooting" : 'Shooting Total', 'physic' : 'Physicality Total',
                          'passing' : "Passing Total"},inplace=True)
    #first we need to check the team names
    fifa_team_names = curr_players['Club Name'].values
    checked = []
    not_found = []
    names_fix = {'Celta Vigo': 'RC Celta de Vigo'
                 , 'Atlético Madrid': 'Atlético de Madrid'
                 , 'Bayern Munich': 'FC Bayern München'
                 , 'Eint Frankfurt': 'Eintracht Frankfurt'
                 , 'Gladbach': 'Borussia Mönchengladbach',
                 'Sporting Gijón':'Real Sporting de Gijón'}
    for team in curr_df['home_team_name'].values:
        
        if (team not in fifa_team_names and team not in checked) and team not in names_fix.keys():
            checked.append(team)
            #print(team+" not found")
            
            alt = find_closest_name(team, fifa_team_names)
            if(alt):
                print(team+" can be replaced with "+alt)
                names_fix[team] = alt
            else:
                not_found.append(team)

    print(not_found)
    for key in names_fix.keys():
        curr_players['Club Name'].replace(names_fix[key],key, inplace=True)
    season = pipeline(curr_df, curr_players)
    season.to_csv("season_data/season"+year+'.csv')
    

C:\Users\Yuval\AppData\Local\Temp\ipykernel_106068\1333768337.py:16: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  curr_players = pd.read_csv('players_'+year[-2:]+'.csv')


Málaga can be replaced with Málaga CF
Granada can be replaced with Granada CF
Sevilla can be replaced with Sevilla FC
Almería can be replaced with Unión Deportiva Almería
Eibar can be replaced with SD Eibar
Barcelona can be replaced with FC Barcelona
Levante can be replaced with Levante Unión Deportiva
Real Madrid can be replaced with Real Madrid CF
Getafe can be replaced with Getafe CF
Valencia can be replaced with Valencia CF
Córdoba can be replaced with Córdoba CF
Athletic Club can be replaced with Athletic Club de Bilbao
Espanyol can be replaced with RCD Espanyol de Barcelona
Villarreal can be replaced with Villarreal CF
La Coruña can be replaced with Deportivo de La Coruña
Elche can be replaced with Elche CF
Chievo can be replaced with Chievo Verona
Milan can be replaced with AC Milan
Torino can be replaced with Torino F.C.
Sassuolo can be replaced with U.S. Sassuolo Calcio
Udinese can be replaced with Udinese Calcio
Sampdoria can be replaced with U.C. Sampdoria
Hoffenheim can be 

C:\Users\Yuval\AppData\Local\Temp\ipykernel_106068\1333768337.py:55: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  curr_players['Club Name'].replace(names_fix[key],key, inplace=True)


found player Vicente Guaita Panadero with a score of 0.7567567567567568
found player Guaita with a score of 0.6
found player Vicente Guaita Panadero with a score of 0.7567567567567568
found player Guaita with a score of 0.6
found player Diego Alves Carreira with a score of 0.7096774193548387
found player Diego Alves with a score of 1.0
found player Diego Alves Carreira with a score of 0.7096774193548387
found player Diego Alves with a score of 1.0
found player Juan Carlos Martín Corral with a score of 0.6111111111111112
found player Juan Carlos with a score of 1.0
found player Juan Carlos Martín Corral with a score of 0.6111111111111112
found player Juan Carlos with a score of 1.0
found player Gorka Iraizoz Moreno with a score of 0.7878787878787878
found player Iraizoz with a score of 0.7
found player Gorka Iraizoz Moreno with a score of 0.7878787878787878
found player Iraizoz with a score of 0.7
found player Miguel Ángel Moyá Rumbo with a score of 0.85
found player Moyá with a score o

KeyboardInterrupt: 

In [ ]:
### fixing columns

for year in years:
    curr = pd.read_csv('season_data/season'+year+'.csv')
    for col in curr.columns:
        if ('Home' in col or 'Away' in col) and 'Overall' not in col:
            curr.drop(col, axis=1, inplace=True)
    curr.drop('matchweek_num', axis=1,inplace=True)
    curr.rename(columns = {'matchweek' : 'Matchweek'},inplace=True)
    epl_curr = pd.read_csv("epl_data(fully_proccessed)/epl"+year+"_proccessed_with_form.csv")
    epl_curr = epl_curr[curr.columns]
    curr = pd.concat([curr, epl_curr],axis=0, ignore_index=True)
    curr.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace=True)
    curr.to_csv("season_data/season"+year+"_proccessed.csv")